<a href="https://colab.research.google.com/github/KhushbooChoudhary11/RecommendationSystem/blob/main/Recommendationsystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import numpy as np
import pandas as pd

In [40]:
books = pd.read_csv('Books.csv')
users = pd.read_csv('Users.csv')
ratings = pd.read_csv('Ratings.csv')

<ipython-input-40-1bb591fbb88d>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv')


In [41]:
books['Image-URL-M'][1]

'http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg'

In [42]:
users.head()

User-ID                            Location   Age
0        1                  nyc, new york, usa   NaN
1        2           stockton, california, usa  18.0
2        3     moscow, yukon territory, russia   NaN
3        4           porto, v.n.gaia, portugal  17.0
4        5  farnborough, hants, united kingdom   NaN

In [43]:
ratings.head()

User-ID        ISBN  Book-Rating
0   276725  034545104X            0
1   276726  0155061224            5
2   276727  0446520802            0
3   276729  052165615X            3
4   276729  0521795028            6

In [44]:
print(books.shape)
print(ratings.shape)
print(users.shape)

(271360, 8)
(1149780, 3)
(278858, 3)


In [45]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [46]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [9]:
ratings.isnull().sum()

User-ID        0
ISBN           1
Book-Rating    1
dtype: int64

In [10]:
books.duplicated().sum()

0

In [11]:
ratings.duplicated().sum()

0

In [12]:
users.duplicated().sum()

0

Popularity Based Recommender System

In [13]:
ratings_with_name = ratings.merge(books,on='ISBN')

In [14]:
num_rating_df = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)
num_rating_df

Book-Title  num_ratings
0        A Light in the Storm: The Civil War Diary of ...            4
1                                   Always Have Popsicles            1
2        Beyond IBM: Leadership Marketing and Finance ...            1
3                                            Dark Justice            1
4                                                Deceived            1
...                                                   ...          ...
197746                   Ã?Â?berraschung am Valentinstag.            1
197747                                      Ã?Â?lpiraten.            2
197748                     Ã?Â?rger mit Produkt X. Roman.            3
197749                              Ã?Â?stlich der Berge.            1
197750                                  Ã?Â?thique en toc            2

[197751 rows x 2 columns]

In [16]:
avg_rating_df = ratings_with_name.groupby('Book-Title')['Book-Rating'].mean(numeric_only=True).reset_index()
avg_rating_df.rename(columns={'Book-Rating':'avg_rating'},inplace=True)
avg_rating_df

Book-Title  avg_rating
0        A Light in the Storm: The Civil War Diary of ...    2.250000
1                                   Always Have Popsicles    0.000000
2        Beyond IBM: Leadership Marketing and Finance ...    0.000000
3                                            Dark Justice   10.000000
4                                                Deceived    0.000000
...                                                   ...         ...
197746                   Ã?Â?berraschung am Valentinstag.    0.000000
197747                                      Ã?Â?lpiraten.    0.000000
197748                     Ã?Â?rger mit Produkt X. Roman.    4.666667
197749                              Ã?Â?stlich der Berge.    8.000000
197750                                  Ã?Â?thique en toc    4.000000

[197751 rows x 2 columns]

In [17]:
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df

Book-Title  num_ratings  \
0        A Light in the Storm: The Civil War Diary of ...            4   
1                                   Always Have Popsicles            1   
2        Beyond IBM: Leadership Marketing and Finance ...            1   
3                                            Dark Justice            1   
4                                                Deceived            1   
...                                                   ...          ...   
197746                   Ã?Â?berraschung am Valentinstag.            1   
197747                                      Ã?Â?lpiraten.            2   
197748                     Ã?Â?rger mit Produkt X. Roman.            3   
197749                              Ã?Â?stlich der Berge.            1   
197750                                  Ã?Â?thique en toc            2   

        avg_rating  
0         2.250000  
1         0.000000  
2         0.000000  
3        10.000000  
4         0.000000  
...            ...  
197746    0.000000  
197747    0.000000  
197748    4.666667  
197749    8.000000  
197750    4.000000  

[197751 rows x 3 columns]

In [18]:
popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg_rating',ascending=False).head(50)

In [19]:
popular_df = popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg_rating']]

In [20]:
popular_df['Image-URL-M'][0]

'http://images.amazon.com/images/P/0439139597.01.MZZZZZZZ.jpg'

Collaborative Filtering Based Recommender System

In [21]:
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200
padhe_likhe_users = x[x].index

In [22]:
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(padhe_likhe_users)]

In [23]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index

In [24]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [25]:
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [26]:
pt.fillna(0,inplace=True)

In [27]:
pt

User-ID                                             254     2276    2766    \
Book-Title                                                                   
1984                                                   9.0     0.0     0.0   
1st to Die: A Novel                                    0.0     0.0     0.0   
2nd Chance                                             0.0    10.0     0.0   
A Bend in the Road                                     0.0     0.0     7.0   
A Child Called \It\": One Child's Courage to Su...     0.0     0.0     0.0   
...                                                    ...     ...     ...   
Wicked: The Life and Times of the Wicked Witch ...     0.0     0.0     0.0   
Wild Animus                                            0.0     0.0     6.0   
Wish You Well                                          0.0     0.0     0.0   
Wuthering Heights                                      0.0     0.0     0.0   
\O\" Is for Outlaw"                                    0.0     0.0     0.0   

User-ID                                             2977    3363    4017    \
Book-Title                                                                   
1984                                                   0.0     0.0     0.0   
1st to Die: A Novel                                    0.0     0.0     0.0   
2nd Chance                                             0.0     0.0     0.0   
A Bend in the Road                                     0.0     0.0     0.0   
A Child Called \It\": One Child's Courage to Su...     0.0     0.0     0.0   
...                                                    ...     ...     ...   
Wicked: The Life and Times of the Wicked Witch ...     0.0     0.0     0.0   
Wild Animus                                            0.0     0.0     0.0   
Wish You Well                                          0.0     0.0     0.0   
Wuthering Heights                                      0.0     0.0     0.0   
\O\" Is for Outlaw"                                    0.0     0.0     0.0   

User-ID                                             4385    6251    6323    \
Book-Title                                                                   
1984                                                   0.0     0.0     0.0   
1st to Die: A Novel                                    0.0     0.0     0.0   
2nd Chance                                             0.0     0.0     0.0   
A Bend in the Road                                     0.0     0.0     0.0   
A Child Called \It\": One Child's Courage to Su...     0.0     0.0     0.0   
...                                                    ...     ...     ...   
Wicked: The Life and Times of the Wicked Witch ...     0.0     0.0     0.0   
Wild Animus                                            0.0     0.0     0.0   
Wish You Well                                          0.0     0.0     0.0   
Wuthering Heights                                      0.0     0.0     0.0   
\O\" Is for Outlaw"                                    0.0     0.0     0.0   

User-ID                                             6543    ...  190925  \
Book-Title                                                  ...           
1984                                                   0.0  ...     0.0   
1st to Die: A Novel                                    9.0  ...     8.0   
2nd Chance                                             0.0  ...     0.0   
A Bend in the Road                                     0.0  ...     0.0   
A Child Called \It\": One Child's Courage to Su...     0.0  ...     0.0   
...                                                    ...  ...     ...   
Wicked: The Life and Times of the Wicked Witch ...    10.0  ...     0.0   
Wild Animus                                            0.0  ...     0.0   
Wish You Well                                          0.0  ...     0.0   
Wuthering Heights                                      0.0  ...     0.0   
\O\" Is for Outlaw"                                    

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
similarity_scores = cosine_similarity(pt)

In [30]:
similarity_scores.shape

(342, 342)

In [31]:
def recommend(book_name):
    # index fetch
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:5]

    data = []
    for i in similar_items:
        item = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))

        data.append(item)

    return data

In [32]:
recommend('1984')

[['The Tale of the Body Thief (Vampire Chronicles (Paperback))',
  'Anne Rice',
  'http://images.amazon.com/images/P/034538475X.01.MZZZZZZZ.jpg'],
 ['Fahrenheit 451',
  'Ray Bradbury',
  'http://images.amazon.com/images/P/3257208626.01.MZZZZZZZ.jpg'],
 ['Brave New World',
  'Aldous Huxley',
  'http://images.amazon.com/images/P/0060809833.01.MZZZZZZZ.jpg'],
 ["The Bonesetter's Daughter",
  'Amy Tan',
  'http://images.amazon.com/images/P/0399146431.01.MZZZZZZZ.jpg']]

In [34]:
pt.index[333]

'While My Pretty One Sleeps'

In [35]:
import pickle
pickle.dump(popular_df,open('popular.pkl','wb'))